In [ ]:
#!import "../DataModel/DataStructure"

In [ ]:
using System ;
using System.Diagnostics ; 
using System.Threading ;

In [ ]:
using static Systemorph.Vertex.Equality.IdentityPropertyExtensions;

public class MyClass{
    [IdentityProperty]
    public string Prop1 {get; set;}
    [IdentityProperty]
    public string Prop2 {get; set;}
}

In [ ]:
var a = new MyClass[] { new MyClass{Prop1="blabla", Prop2="blublu"}, new MyClass{Prop1="bloblo", Prop2="blublu"}, new MyClass{Prop1="blibli", Prop2="blublu"}, new MyClass{Prop1="blibli", Prop2="blublu"} };

In [ ]:
Stopwatch sw = new Stopwatch();
sw.Start();
var r = new List<string>();
for(int i = 0; i < 10000000; i++) {
    for(int j = 0; j < a.Length; j++) {
        r.Add((string)typeof(MyClass).GetIdentityProperties().First().GetValue(a[j]));
    }
}
sw.Stop();
sw.Elapsed

In [ ]:
Stopwatch sw = new Stopwatch();
sw.Start();
var r = new List<string>();
for(int i = 0; i < 10000000; i++) {
    for(int j = 0; j < a.Length; j++) {
        r.Add(a[j].Prop1);
    }
}
sw.Stop();
sw.Elapsed

In [ ]:
class RawVariableComparer: IEqualityComparer<RawVariable>
{
    private bool IgnoreValues;
    private RawVariableComparer(bool ignoreValues)
    {
        IgnoreValues = ignoreValues;
    }

    public bool Equals(RawVariable x, RawVariable y) =>
        x.AccidentYear == y.AccidentYear && x.AmountType == y.AmountType && x.DataNode == y.DataNode && x.AocType == y.AocType && 
        x.Novelty == y.Novelty && x.EstimateType == y.EstimateType && (IgnoreValues ? true : x.Values.SequenceEqual(y.Values)); // TODO: enable Precision

    public int GetHashCode(RawVariable v) => IgnoreValues  
                                            ? HashCode.Combine(v.AccidentYear, v.AmountType, v.DataNode, v.AocType, v.Novelty) 
                                            : HashCode.Combine(v.AccidentYear, v.AmountType, v.DataNode, v.AocType, v.Novelty, v.Values);

    public static RawVariableComparer Instance(bool ignoreValues = false)
    { 
        return new RawVariableComparer(ignoreValues);
    }
}

In [ ]:
class IfrsVariableComparer: IEqualityComparer<IfrsVariable>
{
    private bool IgnoreValue;
    private IfrsVariableComparer(bool ignoreValue)
    {
        IgnoreValue = ignoreValue;
    }

    public bool Equals(IfrsVariable x, IfrsVariable y) =>
        x.AccidentYear == y.AccidentYear && x.AmountType == y.AmountType && x.DataNode == y.DataNode && x.AocType == y.AocType && 
        x.Novelty == y.Novelty && x.EstimateType == y.EstimateType && (IgnoreValue ? true : Math.Abs(x.Value - y.Value) < Precision); 

    public int GetHashCode(IfrsVariable v) => IgnoreValue  
                                            ? HashCode.Combine(v.AccidentYear, v.AmountType, v.DataNode, v.AocType, v.Novelty) 
                                            : HashCode.Combine(v.AccidentYear, v.AmountType, v.DataNode, v.AocType, v.Novelty, v.Value.ToString("F5"));

    public static IfrsVariableComparer Instance(bool ignoreValue = false)
    { 
        return new IfrsVariableComparer(ignoreValue);
    }
}

In [ ]:
class YieldCurveComparer: IEqualityComparer<YieldCurve>
{
    private YieldCurveComparer(){}

	public bool Equals(YieldCurve x, YieldCurve y) => 
            x.Year == y.Year && x.Month == y.Month && x.Scenario == y.Scenario && x.Currency == y.Currency && x.Id == y.Id && x.Values.SequenceEqual(y.Values); // TODO: enable Precision
	
	public int GetHashCode (YieldCurve x) => 0;

    public static YieldCurveComparer Instance() => new YieldCurveComparer();
}


In [ ]:
using System;
using System.Collections.Generic;
using System.Diagnostics; 
using System.Linq;
using System.Linq.Expressions;
using System.Reflection;
using System.Threading;
//using Systemorph.Domain;
//using Systemorph.Utils.Reflection;
using static Systemorph.Vertex.Equality.IdentityPropertyExtensions;

In [ ]:
class EqualityComparer<T> : IEqualityComparer<T>
{
    private static readonly System.Reflection.PropertyInfo[] IdentityProperties = typeof(T).GetIdentityProperties().ToArray();
    private static Func<T, T, bool> compiledEqualityFunction;

    private EqualityComparer() {
        compiledEqualityFunction = GetEqualityFunction();
    }

    public static readonly EqualityComparer<T> Instance = new EqualityComparer<T>();

    public bool Equals(T x, T y) => compiledEqualityFunction(x, y);
    public int GetHashCode(T obj) => 0;

    private static Func<T, T, bool> GetEqualityFunction()
    {
        var prm1 = Expression.Parameter(typeof(T));
        var prm2 = Expression.Parameter(typeof(T));

        // r1 == null && r2 == null
        var nullConditionExpression = Expression.AndAlso(Expression.Equal(prm1, Expression.Constant(null, typeof(T))), Expression.Equal(prm2, Expression.Constant(null, typeof(T))));
        // r1 != null && r2 != null
        var nonNullConditionExpression = Expression.AndAlso(Expression.NotEqual(prm1, Expression.Constant(null, typeof(T))), Expression.NotEqual(prm2, Expression.Constant(null, typeof(T))));
        // r1.prop1 == r2.prop1 && r1.prop2 == r2.prop2...... 
        var allPropertiesEqualExpression = IdentityProperties.Select(propertyInfo => Expression.Equal(Expression.Property(prm1, propertyInfo), Expression.Property(prm2, propertyInfo))).Aggregate(Expression.AndAlso);

        var equalityExpr = Expression.OrElse(nullConditionExpression, Expression.AndAlso(nonNullConditionExpression, allPropertiesEqualExpression));
        return Expression.Lambda<Func<T, T, bool>>(equalityExpr, prm1, prm2).Compile();
    }
}